In [3]:
import requests

## Esta librería no te servirá en este análisis, pero
## es la librería básica para scrapear páginas más sencillas

from bs4 import BeautifulSoup as bs

## Esta sí. Con ella, sacaremos datos del HTML.

import random

## Para usar números aleatorios. Esta no era tan complicada.
## La usaremos para no usar el mismo tiempo siempre al scrapear.

import time

## Con esta metemos el retardo a la máquina. Las máquinas van rápidas. Yo no.

import pandas as pd
import numpy as np

## EL bread and butter del Data Science. Tablas y operaciones aritméticas.

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## Esta es la manera para usar selenium normal, pero nosotros tenemos que parecer humanos.
## Por eso, usamos la siguiente librería.

import undetected_chromedriver as uc

## La persona que desarrolló esto es un crack. Configuró todos los proxies y
## demás (digo demás porque no se qué brujería ha usado) para ser indetectable.
## Usaremos este browser.

In [5]:
browser = uc.Chrome()


In [6]:
x=1
ids = []

while True:
    
    url = f'https://www.idealista.com/venta-viviendas/madrid-madrid/pagina-{x}.htm'
    
    browser.get(url)
    
    time.sleep(random.randint(10,12))
    
    try:
        
        browser.find_element("xpath",'//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass
    
    html = browser.page_source
    
    soup = bs(html, 'lxml')
    
    pagina_actual = int(soup.find('main',{'class':'listing-items'}).find('div',{'class':'pagination'}).find('li',{'class':'selected'}).text)
    
    if x == pagina_actual:

        articles = soup.find('main',{'class':'listing-items'}).find_all('article')

    else:
        break
    
    x = x+1
    
    for article in articles:
        
        id_muebles = article.get('data-element-id')
        
        ids.append(id_muebles)
        
        time.sleep(random.randint(1,3))
        print(id_muebles)
    
    ids = [muebles for muebles in ids if muebles is not None]

97875752
104952998
106107690


KeyboardInterrupt: 

In [7]:
ids_casas = pd.DataFrame(ids)
ids_casas.columns = ['id']


In [8]:
ids_casas.to_csv('ids_casas.csv',index = False)

In [9]:
casas = pd.Series()

In [10]:
def parsear_inmueble(id_inmueble):
    
    print( '\n Casa numero: ' + id_inmueble)
    
    url = "https://www.idealista.com/inmueble/" + id_inmueble + "/"
    
    browser.get(url)

    html = browser.page_source
    
    soup = bs(html, 'lxml')

    titulo = soup.find('span',{'class':'main-info__title-main'}).text
    
    print('\n Titulo: ' + titulo)
    
    localizacion = soup.find('span',{'class':'main-info__title-minor'}).text.split(',')[0]

    print('\n Localizacion: ' + localizacion)
    precio = int(soup.find('span',{'class':'txt-bold'}).text.replace('.',''))

    c1 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-one'})

    caract_basicas = [caract.text.strip() for caract in c1.find_all('li')]
    
    #print('Caracteristicas basicas:' + caract_basicas)

    c2 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-two'})

    caract_extra = [caract.text.strip() for caract in c2.find_all('li')]
    
    #print('Caracteristicas extras:' + caract_extra)
    
    casas['titulo'] = titulo
    
    casas['localizacion'] = localizacion
    
    casas['precio'] = precio
    
    casas['caracteristicas_basicas'] = caract_basicas
    
    casas['caracteristicas_extras'] = caract_extra
    
    df_casas = pd.DataFrame(casas)
    
    return(df_casas.T)

In [11]:
soup

<html class="" data-userauth="false" env="es" lang="es" username=""><head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Casas y pisos en Madrid — idealista</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="13.986 anuncios de casas y pisos en Madrid, a partir de 12.000 euros. Casas y pisos en Madrid de particulares y agencias inmobiliarias." name="description"/>
<meta content="idealista.com" name="author"/>
<meta content="on" http-equiv="cleartype"/>
<meta content="no-cache" name="pragma"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="Fri, 01 Jan 1990 00:00:00 GMT" http-equiv="Expires"/>
<meta content="no-cache, no-store, must-revalidate" http-equiv="Cache-Control"/>
<link href="https://st3.idealista.com/static/common/icons/favicon.ico?20241025074738" rel="shortcut icon"/>
<link href="https://st3.idealista.com/static/common/icons/32x32.png?20241025074738" rel="apple-touch-icon" sizes="32x32"/>
<link hre

In [22]:
df_casas = parsear_inmueble(ids_casas.iloc[0].id)

for i in range(1,len(ids)):
    
    df_casas = pd.concat([df_casas,parsear_inmueble(ids[i])])
    
    time.sleep(random.randint(4,8))


 Casa numero: 97875752

 Titulo:  Chalet adosado en venta en paseo de La Habana

 Localizacion: Nueva España

 Casa numero: 104952998

 Titulo: Piso en venta en calle del Duque de Sesto

 Localizacion: Goya

 Casa numero: 106107690

 Titulo: Ático en venta en Adelfas

 Localizacion: Retiro

 Casa numero: 103910133

 Titulo:  Casa o chalet independiente en venta en calle de Darío Aparicio

 Localizacion: Valdemarín

 Casa numero: 101792453

 Titulo:  Chalet pareado en venta en calle Virgen de los Rosales

 Localizacion: Aravaca

 Casa numero: 106079633

 Titulo: Piso en venta en calle de Lagasca

 Localizacion: Castellana

 Casa numero: 105893822

 Titulo:  Chalet pareado en venta en calle Virgen de los Rosales

 Localizacion: Aravaca

 Casa numero: 106143023

 Titulo: Piso en venta en calle de Núñez de Balboa

 Localizacion: Castellana

 Casa numero: 104938362

 Titulo: Piso en venta en calle del Duque de Sesto

 Localizacion: Goya

 Casa numero: 105891810

 Titulo:  Chalet adosado en

KeyboardInterrupt: 

In [20]:
df_casas.reset_index(drop=True, inplace=True)

In [21]:
df_casas.to_csv('casas_idealista.csv', index = False, sep = ';', encoding = 'utf-16')